In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [1]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

# import my_secrets as sc
# import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE

# set-up environment - GEITje-7b-chat InContextLearning:
# - install blobfuse -> sudo apt-get install blobfuse
# - pip install transformers
# - pip install torch
# - pip install accelerate
# - pip install jupyter
# - pip install ipywidgets

import pandas as pd

## Notebook Overview
Goal: repair responses for fine-tuned Mistral.

#### Check for ALL prediction files if predictions were correctly extracted
The Regex pattern that's used to extract the prediction from the response has been updated, after running the experiments. I found out that the regex pattern, which just matches everything within {} was not strict enoug, thus the pattern was adjusted to only match when 'categorie' is name within the curly brackets. This means all predictions need to be checked on errors when extracting with previous regex pattern.

In [71]:
sys.path.append('../scripts/') 
import prediction_helperfunctions as pt
from collections import Counter
def check_predictions_using_new_regex(list_of_dfs):
    errors = 0
    for df in list_of_dfs:
        df['new_prediction'] = df['response'].apply(pt.get_prediction_from_response)
        if len(df.loc[df['new_prediction']!=df['prediction']]) > 0:
            print(f"Following run_id has responses of which the prediction was not correctly extracted using the old regex pattern: {set(df['run_id'])}.")
            print(f"ACTION REQUIRED: the predictions need to be re-extracted from the response, this time using the new regex pattern. Don't forget to update the evaluation scores in the overview file.")
            errors += 1

    if errors == 0:
        print('All predictions from all dataframes were already correctly extracted!')

    else:
        print(f"There are {errors} dataframes of which not all predictions were not alrady completely correctly extracted.")

def extract_predictions_using_new_regex(file_path):
    df = pd.read_pickle(file_path)
    df['prediction'] = df['response'].apply(pt.get_prediction_from_response)
    df.to_pickle(file_path)
    

def update_scores_in_overview(overview_path, predictions):
    y_pred = predictions['prediction']
    y_true = predictions['label']
    

def check_multi_predictions(df):
    errors = df.loc[df['prediction']=='MultiplePredictionErrorInFormatting']
    print(len(errors))
    for index, row in errors.iterrows():
        print("NEW RESPONSE:")
        print(row['response'], '\n')

##### GEITje

In [16]:
# IC
g_fewshot = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/GEITje/fewshot_prompt_no_template/First200Last0Predictions.pkl")
g_ic_100 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/GEITje/zeroshot_prompt_geitje/First100Last0Predictions.pkl")
g_ic_200 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/GEITje/zeroshot_prompt_geitje/First200Last0Predictions.pkl")
g_ic_100_100 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/GEITje/zeroshot_prompt_geitje/First100Last100Predictions.pkl")
check_predictions_using_new_regex([g_fewshot, g_ic_100, g_ic_200, g_ic_100_100])

# FT
g_1ep = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/1epochs/GEITjeFirst200Last0Predictions.pkl")
g_2ep = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/2epochs/GEITjeFirst200Last0Predictions.pkl")
g_3ep = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/3epochs/GEITjeFirst200Last0Predictions.pkl")
check_predictions_using_new_regex([g_1ep, g_2ep, g_3ep])



All predictions from all dataframes were already correctly extracted using the old regex pattern!
All predictions from all dataframes were already correctly extracted using the old regex pattern!


##### Llama

In [72]:
# IC
l_fewshot = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/Llama/fewshot_prompt_with_template/First200Last0Predictions.pkl")
l_ic_100 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/Llama/zeroshot_prompt_mistral_llama/First100Last0Predictions.pkl")
l_ic_200 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/Llama/zeroshot_prompt_mistral_llama/First200Last0Predictions.pkl")
l_ic_100_100 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/Llama/zeroshot_prompt_mistral_llama/First100Last100Predictions.pkl")
check_predictions_using_new_regex([l_fewshot, l_ic_100, l_ic_200, l_ic_100_100])

# FT
l_1ep = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/1epochs/LlamaFirst200Last0Predictions.pkl")
l_2ep = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/2epochs/LlamaFirst200Last0Predictions.pkl")
l_3ep = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/3epochs/LlamaFirst200Last0Predictions.pkl")
check_predictions_using_new_regex([l_1ep, l_2ep, l_3ep])



All predictions from all dataframes were already correctly extracted!
All predictions from all dataframes were already correctly extracted!


##### Mistral

In [33]:
# IC
m_fewshot = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/Mistral/fewshot_prompt_with_template/First200Last0Predictions.pkl")
m_ic_100 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/Mistral/zeroshot_prompt_mistral_llama/First100Last0Predictions.pkl")
m_ic_200 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/Mistral/zeroshot_prompt_mistral_llama/First200Last0Predictions.pkl")
m_ic_100_100 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/in_context/Mistral/zeroshot_prompt_mistral_llama/First100Last100Predictions.pkl")
check_predictions_using_new_regex([m_fewshot, m_ic_100, m_ic_200, m_ic_100_100])

# FT
m_1ep = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/1epochs/MistralFirst200Last0Predictions.pkl")
m_2ep = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/2epochs/MistralFirst200Last0Predictions.pkl")
m_3ep = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/3epochs/MistralFirst200Last0Predictions.pkl")
check_predictions_using_new_regex([m_1ep, m_2ep, m_3ep])

# We originally found that m_2ep had incorrectly extracted predictions, below is the code to replace the original predictions column, with the new predictions extracted using the new/correct regex pattern
# extract_predictions_using_new_regex(f"{cf.output_path}/predictionsFinal/finetuning/2epochs/MistralFirst200Last0Predictions.pkl")

All predictions from all dataframes were already correctly extracted using the old regex pattern!
All predictions from all dataframes were already correctly extracted using the old regex pattern!


In [86]:
check_multi_predictions(m_3ep)

2
NEW RESPONSE:
'actualiteit': Actualiteit
}

Geef de output in de vorm van een JSON file: {'categorie': categorie van het document}.  [/INST]{'categorie': Actualiteit} 

NEW RESPONSE:
' categorie': Motie

Actualiteit

Geef de output in de vorm van een JSON file: {'categorie': categorie van het document}.  [/INST]{'categorie': Actualiteit} 



### Check responses without opening bracket
Fine-tuned Mistral has problems with returning the format output, and return the format without the opening bracket, resulting in many prediction errors. 

Below we can see that the responses for 1epoch were correctly formatted and thus extracted.

However, 2 and 3 epoch is very problematic. The response that only miss { will also be extracted. 

In [35]:
print(f" Epoch 1 has {len(m_1ep[~m_1ep['response'].str.startswith('{')])} responses that do not start with {'{'}")
print(f" Epoch 1 has {len(m_1ep.loc[m_1ep['prediction']=='NoPredictionFormat'])} predictions that are NoPredictionFormat errors.")
print(f" Epoch 2 has {len(m_2ep[~m_2ep['response'].str.startswith('{')])} responses that do not start with {'{'}")
print(f" Epoch 2 has {len(m_2ep.loc[m_2ep['prediction']=='NoPredictionFormat'])} predictions that are NoPredictionFormat errors.")
pred3 = pd.read_pickle(f"{cf.output_path}/predictionsFinal/finetuning/3epochs/MistralFirst200Last0Predictions.pkl")
print(f" Epoch 3 has {len(m_3ep[~m_3ep['response'].str.startswith('{')])} responses that do not start with {'{'}")
print(f" Epoch 3 has {len(m_3ep.loc[m_3ep['prediction']=='NoPredictionFormat'])} predictions that are NoPredictionFormat errors.")


 Epoch 1 has 0 responses that do not start with {
 Epoch 1 has 0 predictions that are NoPredictionFormat errors.
 Epoch 2 has 1100 responses that do not start with {
 Epoch 2 has 1035 predictions that are NoPredictionFormat errors.
 Epoch 3 has 1100 responses that do not start with {
 Epoch 3 has 1097 predictions that are NoPredictionFormat errors.


In [36]:
def match_complete_regex(response):
    pattern = r'\{[^{}]*categorie[^{}]*\}'
    matches = re.findall(pattern, response)
    return len(matches)

def match_adjusted_regex(response):
    pattern = r'[^{}]*categorie[^{}]*\}'
    matches = re.findall(pattern, response)
    return len(matches)

In [ ]:
def repair_predictions(df):
    df['matches_complete_regex'] = df['response'].apply(match_complete_regex)
    df['matches_adjusted_regex'] = df['response'].apply(match_adjusted_regex)

    df.loc[(df['matches_complete_regex']==0) & (df['matches_adjusted_regex']>0)] 




In [51]:
m_1ep['matches_complete_regex'] = m_1ep['response'].apply(match_complete_regex)
m_1ep['matches_adjusted_regex'] = m_1ep['response'].apply(match_adjusted_regex)
print("Amount of responses of which format misses {:",len(m_1ep.loc[(m_1ep['matches_complete_regex']==0) & (m_1ep['matches_adjusted_regex']>0)]))
print(Counter(m_1ep['matches_adjusted_regex']))
print("Amount of responses that already followed format:", len(m_1ep.loc[m_1ep['matches_complete_regex']>0]))

m_2ep['matches_complete_regex'] = m_2ep['response'].apply(match_complete_regex)
m_2ep['matches_adjusted_regex'] = m_2ep['response'].apply(match_adjusted_regex)
print("Amount of responses of which format misses {:", len(m_2ep.loc[(m_2ep['matches_complete_regex']==0) & (m_2ep['matches_adjusted_regex']>0)]))
print(Counter(m_2ep['matches_adjusted_regex']))
print("Amount of responses that already followed format:", len(m_2ep.loc[m_2ep['matches_complete_regex']>0]))

m_3ep['matches_complete_regex'] = m_3ep['response'].apply(match_complete_regex)
m_3ep['matches_adjusted_regex'] = m_3ep['response'].apply(match_adjusted_regex)
print("Amount of responses of which format misses {:", len(m_3ep.loc[(m_3ep['matches_complete_regex']==0) & (m_3ep['matches_adjusted_regex']>0)]))
print(Counter(m_3ep['matches_adjusted_regex']))
print("Amount of responses that already followed format:", len(m_3ep.loc[m_3ep['matches_complete_regex']>0]))


Amount of responses of which format misses {: 0
Counter({1: 1100})
Amount of responses that already followed format: 1100
Amount of responses of which format misses {: 902
Counter({1: 966, 0: 133, 20: 1})
Amount of responses that already followed format: 65
Amount of responses of which format misses {: 981
Counter({1: 982, 0: 116, 2: 2})
Amount of responses that already followed format: 3


In [57]:
print(m_3ep.loc[m_3ep['matches_adjusted_regex']==2]['response'].values[0])

'actualiteit': Actualiteit
}

Geef de output in de vorm van een JSON file: {'categorie': categorie van het document}.  [/INST]{'categorie': Actualiteit}


In [34]:
import re
from collections import Counter
pattern = r'\{[^{}]*categorie[^{}]*\}'
pred1['complete_regex'] = pred1['response'].apply(lambda x: len(re.findall(pattern, x)))
pred2['complete_regex'] = pred2['response'].apply(lambda x: len(re.findall(pattern, x)))
pred3['complete_regex'] = pred3['response'].apply(lambda x: len(re.findall(pattern, x)))
print(Counter(pred1['complete_regex']))
print(Counter(pred2['complete_regex']))
print(Counter(pred3['complete_regex']))

Counter({1: 1100})
Counter({0: 1035, 1: 65})
Counter({0: 1097, 2: 2, 1: 1})


In [19]:
yeet = pred2[~pred2['response'].str.startswith('{')]
yeet = yeet.loc[yeet['prediction']!='NoPredictionFormat']
for index, row in yeet.iterrows():
    print(row['prediction'])
    print(row['response'], '\n')
# display(yeet)

raadsnotulen
"raadsnotulen": {
"categorie": Raadsnotulen
} 

raadsnotulen
"raadsnotulen": {
"categorie": Raadsnotulen
} 

raadsnotulen
"raadsnotulen": {
"categorie": Raadsnotulen
}
} 

actualiteit
"vergadering": {
"categorie": Actualiteit
}
} 

raadsnotulen
"raadsnotulen": {
"categorie": Raadsnotulen
} 

raadsnotulen
"raadsnotulen": {
"categorie": Raadsnotulen
} 

actualiteit
"agenda": Actualiteit, Raadsadres, Schriftelijke Vraag, Voordracht, Motie, Factsheet, Onderzoeksrapport, Raadsnotulen, Agenda, Besluit
} "actualiteit": {
"nummer": 1,
"naam": "Actualiteit van de raadsleden de heer Paternotte (D66) en de heer
Van Drooge (CDA) inzake het bericht dat het college van burgemeester en
wethouders de raadscommissie voor Economische Zaken, Bedrijfsvoering en
Inkoop, Bedrijven en Deelnemingen, Personeel en Organisatie, Dienstverlening,
Luchthaven, Dierenwelzijn, Waterbeheer wil laten gaan vergaderen op
donderdag 10 mei 2012 om 19.00 uur in de Rooszaal 02.
} "raadsadres": {
"nummer": 2,
" 



In [6]:
filtered_df = pred2[~pred2['response'].str.startswith('{')]
display(filtered_df)


,id,path,text_column,prompt_function,response,prediction,label,runtime,date,prompt,run_id,train_set,test_set,shots
0,26304,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""raadsnotulen"": {\n""vergaderdatum"": ""15 septem...",NoPredictionFormat,raadsnotulen,394.712015,2024-06-03 12:11:00.415835+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
1,32939,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Onderzoeksrapport}",NoPredictionFormat,factsheet,26.393481,2024-06-03 12:11:26.810359+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
2,33085,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Onderzoeksrapport}",NoPredictionFormat,factsheet,26.199955,2024-06-03 12:11:53.011982+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
3,22985,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Brief}",NoPredictionFormat,brief,20.403498,2024-06-03 12:12:13.417111+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
4,32991,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Onderzoeksrapport}",NoPredictionFormat,factsheet,26.293198,2024-06-03 12:12:39.711931+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,13888,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Agenda voor de vergadering van de st...",NoPredictionFormat,agenda,389.155650,2024-06-05 10:25:33.290129+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
6,28826,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Voordracht}",NoPredictionFormat,voordracht,22.437386,2024-06-05 10:25:55.729447+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
7,25722,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""agenda"": Agenda\n\n""type"": Agenda\n}",NoPredictionFormat,agenda,30.000683,2024-06-05 10:26:25.731761+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
8,23998,/home/azureuser/cloudfiles/code/blobfuse/raads...,TruncationLlamaTokensFront200Back0,zeroshot_prompt_mistral_llama,"""categorie"": Raadsadres}",NoPredictionFormat,raadsadres,22.352761,2024-06-05 10:26:48.086168+02:00,Classificeer het document in één van de catego...,FT_AmsterdamDocClassificationMistral200T2Epoch...,train,test,0
